In [1]:
import os

In [2]:
os.listdir('/Users/shashanksrivastava/Desktop/sks/technomile/data/pdf/images')

['page_2.png',
 'page_3.png',
 'page_1.png',
 'page_4.png',
 '.DS_Store',
 'page_5.png',
 'page_7.png',
 'page_6.png',
 'page_10.png',
 'page_11.png',
 'page_12.png',
 'page_8.png',
 'page_9.png']

In [5]:
len('LLaMA: Open and Efﬁcient Foundation Language Models Hugo Touvron∗, Thibaut Lavril∗, Gautier Izacard∗, Xavier Martinet Marie-Anne Lachaux, Timothee Lacroix, Baptiste Rozière')

172

In [ ]:
 def pipeline_v1_persist(self):
        '''
        Seperate indexes for text and image and will utilize seperate index for query as well
        '''
        client = qdrant_client.QdrantClient(path="./data/index")
        text_store = self._create_vector_stores(client=client, collection_name='text_collection_v1')
        image_store = self._create_vector_stores(client=client, collection_name='image_collection_v1')
        
        storage_context = self._create_storage_context(text_store, image_store)
        
        text_documents = self._return_text_documents()
        img_documents = self._return_img_documents()

        
        _ = MultiModalVectorStoreIndex(text_documents, storage_context=storage_context, embed_model=self.embed_model,image_embed_model=self.image_embed_model)
        _ = MultiModalVectorStoreIndex(img_documents, storage_context=storage_context, embed_model=self.embed_model, image_embed_model=self.image_embed_model)
        
        client.close()
        
        return None 
    
def query_engine_v1(self, query):
        client = qdrant_client.QdrantClient(path="./data/index")
        
        text_store = self._create_vector_stores(client=client, collection_name='text_collection_v1')
        image_store = self._create_vector_stores(client=client, collection_name='image_collection_v1')
                        
        text_index = MultiModalVectorStoreIndex.from_vector_store(
            vector_store=text_store,
            embed_model=self.embed_model,
        )
        
        img_index = MultiModalVectorStoreIndex.from_vector_store(
            vector_store=image_store,
            embed_model=self.image_embed_model,
        )
        
        text_retriever = text_index.as_retriever(similarity_top_k=3)
        text_retrieval_results = text_retriever.retrieve(query)
        
        img_retriever = img_index.as_retriever(image_similarity_top_k=3)
        img_retrieval_results = img_retriever.retrieve(query)
        
        retrieved_texts = [doc.node.text for doc in text_retrieval_results]
        retrieved_images = [doc.node.metadata.get('file_path') for doc in img_retrieval_results]
        
        print("text_retrieval_results:::",retrieved_texts)       
        print("img_retrieval_results:::",retrieved_images) 
        
        client.close()
        
        return retrieved_texts, retrieved_images

In [6]:
!pip3 install easyocr

  Using cached easyocr-1.7.1-py3-none-any.whl (2.9 MB)
  Using cached scikit_image-0.24.0-cp39-cp39-macosx_10_9_x86_64.whl (14.1 MB)
  Using cached shapely-2.0.5-cp39-cp39-macosx_10_9_x86_64.whl (1.5 MB)
  Using cached python_bidi-0.4.2-py2.py3-none-any.whl (30 kB)
  Using cached pyclipper-1.3.0.post5-cp39-cp39-macosx_10_9_x86_64.whl (146 kB)
  Using cached ninja-1.11.1.1-py2.py3-none-macosx_10_9_universal2.macosx_10_9_x86_64.macosx_11_0_arm64.macosx_11_0_universal2.whl (270 kB)
  Using cached opencv-python-headless-4.10.0.84.tar.gz (95.1 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Using cached imageio-2.34.2-py3-none-any.whl (313 kB)
  Using cached tifffile-2024.7.2-py3-none-any.whl (225 kB)
  Using cached lazy_loader-0.4-py3-none-any.whl (12 kB)
  Created wheel for opencv-python-headless: filename=opencv_python_headless-4.10.0.84-cp39-cp39-macosx_10_16_x86_64.whl size=27382440 sha256=437726432c072

In [ ]:
import easyocr

def extract_text_from_image(image_path):
    # Initialize the reader with the desired languages
    reader = easyocr.Reader(['en'])  # You can add more languages like ['en', 'es', 'fr']
    
    # Perform OCR on the image
    result = reader.readtext(image_path)
    
    # Extract text from the result
    text = ' '.join([item[1] for item in result])
    
    return text

# Example usage

image_path = '/Users/shashanksrivastava/Desktop/sks/technomile/data/table_images/llama.pdf_page_7_0.png'  # Update this with your image path
extracted_text = extract_text_from_image(image_path)
print(extracted_text)

In [ ]:

    
def persist():
    client = qdrant_client.QdrantClient(path="./data/index")
    
    
    text_store = QdrantVectorStore(
                    client=client, collection_name='text_collection'
                    )
    image_store = QdrantVectorStore(
        client=client, collection_name="image_collection"
    )
    # embed_model = HuggingFaceEmbedding('openai/clip-vit-large-patch14')
    storage_context = StorageContext.from_defaults(vector_store=text_store, image_store=image_store)
    service_context = ServiceContext.from_defaults(llm=None, embed_model=ClipEmbedding(model_name='ViT-L/14',embed_batch_size=16))
    # service_context = ServiceContext.from_defaults(llm=None, embed_model=embed_model)
    
    text_documents = SimpleDirectoryReader('./data/pdf').load_data()
    img_documets = SimpleDirectoryReader('./data/table_images').load_data()
    

    node_parser = SentenceSplitter(chunk_size=100, chunk_overlap=20)
    nodes = node_parser.get_nodes_from_documents(text_documents)
        
    print("text documents loaded:", len(text_documents))
    print("len nodes:",len(nodes))
    print("immage documents loaded:", len(img_documets))
    # print("nodes:",nodes)
    
    index = MultiModalVectorStoreIndex(nodes+img_documets, storage_context=storage_context, service_context=service_context)
    
    query = 'compare palm and llama'
    
    retriever = index.as_retriever(image_similarity_top_k=3)
    retrieval_results = retriever.retrieve(query)
    
    retrieved_image = []
    for res_node in retrieval_results:
        print('\n Node:',res_node.metadata)
        if isinstance(res_node.node, ImageNode):
            retrieved_image.append(res_node.node.metadata["file_path"])
        else:
            pass
            
    print('images:',retrieved_image)
    
# persist()
    
    

In [7]:
import os

def print_project_structure(base_path):
    # Ensure the base path exists
    if not os.path.exists(base_path):
        print(f"Path '{base_path}' does not exist.")
        return
    
    print(f"Project Structure for {base_path}:")
    print_structure(base_path, "")

def print_structure(base_path, indent):
    # Get list of files and directories in the current path
    items = os.listdir(base_path)
    
    # Iterate over each item
    for item in sorted(items):
        # Construct full path to the item
        item_path = os.path.join(base_path, item)
        
        # Print item name with proper indentation
        print(indent + "├── " + item)
        
        # If item is a directory, recursively print its structure
        if os.path.isdir(item_path):
            print_structure(item_path, indent + "│   ")
        # (Optional) If item is a symbolic link, you may want to handle it differently
        # elif os.path.islink(item_path):
        #     link_target = os.readlink(item_path)
        #     print(indent + "└── " + item + " -> " + link_target)

# Example usage:
base_path = "/Users/shashanksrivastava/Desktop/sks/technomile"
print_project_structure(base_path)

Project Structure for /Users/shashanksrivastava/Desktop/sks/technomile:
├── .DS_Store
├── app.py
├── data
│   ├── .DS_Store
│   ├── images
│   │   ├── llama.pdf_page_1.png
│   │   ├── llama.pdf_page_10.png
│   │   ├── llama.pdf_page_11.png
│   │   ├── llama.pdf_page_12.png
│   │   ├── llama.pdf_page_2.png
│   │   ├── llama.pdf_page_3.png
│   │   ├── llama.pdf_page_4.png
│   │   ├── llama.pdf_page_5.png
│   │   ├── llama.pdf_page_6.png
│   │   ├── llama.pdf_page_7.png
│   │   ├── llama.pdf_page_8.png
│   │   ├── llama.pdf_page_9.png
│   ├── index
│   │   ├── .lock
│   │   ├── collection
│   │   │   ├── image_collection_v1
│   │   │   │   ├── storage.sqlite
│   │   │   ├── image_collection_v3
│   │   │   │   ├── storage.sqlite
│   │   │   ├── text_collection_v1
│   │   │   │   ├── storage.sqlite
│   │   │   ├── text_collection_v3
│   │   │   │   ├── storage.sqlite
│   │   ├── meta.json
│   ├── pdf
│   │   ├── .DS_Store
│   │   ├── llama.pdf
│   ├── table_images
│   │   ├── llama.pdf_page

In [ ]:
import os 
import fitz
from PIL import Image
import logging
from src.utils.table_transformer import detect_and_crop_save_table

logger = logging.getLogger(__name__)

class PdfHandler():
    def __init__(self, base_path='./data'):
        self.base_path = base_path
     
    def _list_pdf_files(self):
        pdf_files = []
        pdf_folder_path = os.path.join(self.base_path, 'pdf')
        for root, dirs, files in os.walk(pdf_folder_path):
            for file in files:
                if file.endswith(".pdf"):
                    pdf_files.append(os.path.join(root, file))
        return pdf_files
    
    def _get_pdf_filename(self, pdf_path):
        return os.path.basename(pdf_path)
    
    def _delete_folders(self):
        try:
            paths = ['./data/images', './data/table_images']
            for path in paths:
                if os.path.exists(path):
                    os.rmdir(path)
                    print(f"Directory '{path}' successfully removed.")
        except OSError as e:
            # Handle potential errors
            print(f"Error: {path} : {e.strerror}")
        
    def _convert_pdf_to_image(self):
        print("Converting PDF pages to images")
        for pdf_path in self._list_pdf_files():
            pdf_document = fitz.open(pdf_path)


            for page_number in range(pdf_document.page_count):
                # Get the page
                page = pdf_document[page_number]

                # Convert the page to an image
                pix = page.get_pixmap()

                # Create a Pillow Image object from the pixmap
                image = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
                
                image_output_folder = f"{self.base_path}/images/"
                
                if not os.path.exists(image_output_folder):
                    os.makedirs(image_output_folder)

                # Save the image
                image.save(f"{image_output_folder}/{self._get_pdf_filename(pdf_path)}_page_{page_number + 1}.png")

            # Close the PDF file
            pdf_document.close()
            
    def _extract_tables_from_images(self):
        print("Extracting tables from images")
        for file_name in os.listdir(os.path.join(self.base_path,'images')):
            if 'png' in file_name:
                file_path = os.path.join(self.base_path, 'images' ,file_name)
                detect_and_crop_save_table(file_path, cropped_table_directory=os.path.join(self.base_path,'table_images/'))
                
    def process_pdfs(self):
        print("Processing PDFs")
        self._delete_folders()
        self._convert_pdf_to_image()
        self._extract_tables_from_images()
        
        

            
